In [36]:
import os
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env file
# openai.api_key = os.environ['OPENAI_API_KEY']

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

## Chat API : OpenAI

Let's start with a direct API calls to OpenAI.

In [ ]:
# gpt-3.5-turbo-0301

llm_model = "gpt-4.1-nano" 

In [28]:
def get_completion(prompt, model=llm_model):
    messages = [
        {"role": "developer", "content": "Talk like a pirate."},
        {"role": "user", "content": prompt}
    ]
    completion = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return completion.choices[0].message.content

In [29]:
get_completion("What is 1+1?")

"Arrr, matey! That be a simple one—1 plus 1 be 2, aye! Yarrr, math be as easy as plunderin' a merchant ship!"

In [30]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

style = """American English \
in a calm and respectful tone
"""

In [31]:
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks 
into a style that is American English in a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [32]:
response = get_completion(prompt)

In [37]:
response

"Certainly! Here's the translation in a calm and respectful American English tone:\n\nI'm quite upset that my blender lid came off and splattered smoothie all over my kitchen walls. To make matters worse, the warranty doesn't cover the cleaning costs. I would appreciate your assistance right now."

In [47]:
from langchain.chat_models import init_chat_model

In [53]:
chat_model = init_chat_model(
    llm_model,
    temperature=0.0,
    timeout=10,
    max_tokens=100
)

In [54]:
chat_model

ChatOpenAI(profile={'max_input_tokens': 1047576, 'max_output_tokens': 32768, 'text_inputs': True, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'text_outputs': True, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}, client=<openai.resources.chat.completions.completions.Completions object at 0x113da5cd0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x113c0efd0>, root_client=<openai.OpenAI object at 0x113f40b90>, root_async_client=<openai.AsyncOpenAI object at 0x113c0ed50>, model_name='gpt-4.1-nano', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), request_timeout=10.0, stream_usage=True, max_tokens=100)

In [55]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [57]:
from langchain_core.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(template_string)

In [63]:
customer_style = """American English \
in a calm and respectful tone
"""

In [64]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [68]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [71]:
print(customer_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone
. text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [73]:
customer_response = chat_model.invoke(customer_messages)

In [74]:
customer_response.content

"I'm very upset that my blender lid came off and splattered my kitchen walls with smoothie. To make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I would appreciate your help right now."

In [75]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [76]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [77]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply
)
print(service_messages[0].content)
service_response = chat_model.invoke(service_messages)
service_response.content

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



"Ahoy, valued customer! I be sorry to inform ye that the warranty does not cover the cleaning costs for yer kitchen, as it appears the mishap occurred due to yer own oversight in forgettin' to secure the lid on yer blender before usin' it. Aye, 'tis a tough break, indeed. Fair winds and farewell!"

# Output Parsers

In [78]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [79]:
review_prompt_template = ChatPromptTemplate.from_template(review_template)

In [81]:
messages = review_prompt_template.format_messages(text=customer_review)
response = chat_model.invoke(messages)
print(response.content)

{
  "gift": false,
  "delivery_days": 2,
  "price_value": ["It's slightly more expensive than the other leaf blowers out there"]
}


In [94]:
from pydantic import BaseModel, Field
from langchain.agents import create_agent

class GiftInfo(BaseModel):
    """Contact information for a person."""
    gift: str = Field(description="Was the item purchased as a gift for someone else? \
                                Answer True if yes,False if not or unknown.", )
    delivery_days: str = Field(description="How many days did it take for the product to arrive? \
                                            If this information is not found, output -1.")
    price_value: str = Field(description="Extract any sentences about the value or price, \
        and output them as a string.")

agent = create_agent(
    model=llm_model,
    response_format=GiftInfo  # Auto-selects ProviderStrategy
)

In [95]:
result = agent.invoke({
    "messages": [{"role": "user", 
    "content": f"""
    Extract gift info from the following text:
    
    {customer_review}
    """
    }]
})
print(result["structured_response"])

gift='True' delivery_days='2' price_value="slightly more expensive than the other leaf blowers out there, but I think it's worth it"


In [97]:
result["structured_response"]

GiftInfo(gift='True', delivery_days='2', price_value="slightly more expensive than the other leaf blowers out there, but I think it's worth it")